# Multithreaded Cityblock distance matrix function with SciPy and Dask's delayed execution

In this notebook we the function `scipy.spatial.distance.cdist` to compute the cityblock distance matrix. Althought this function is quite fast, it uses a single thread.
In cases like this one, it might be convenient to implement a multithreaded version of the function by parallelicing the execution over chunks of data.

We have already written the chunk-based computation on this notebook, but it is missing the parallelization. This chunk-based calculation is kind of pointless wihtout the parallelization: Use `dask.delayed` to compute all chunks in parallel and speed up the calculation.

The notebook has no indications of where the modifications need to be done. Just follow the cells and identify what needs to be changed.

In [1]:
import numpy as np
from scipy.spatial.distance import cdist
from dask import compute, delayed, visualize

In [2]:
nsamples = 12000
nfeat = 50

x = 10. * np.random.random([nsamples, nfeat])

Let's time the `cdist` function and look the `top` command.

In [3]:
# observe here that the funcion `cdist` used to get the cityblock distance
# is not multithreaded

%timeit cdist(x, x, 'cityblock')

3.96 s ± 263 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


With the `top` command we see that `cdist` runs in a single thread. In such cases it could be quite simple write a distributed version of the function. We can do this very easily with `dask.delayed`!

## Dask's async delayed execution
A simple distributed version of `cdist` can be done as the following:
  * Split the array of vectors into chunks. We can use `np.split(x, num_chunks)`
  * Compute partial cityblock distance matrices of the complete array with respect to each of the chunks
  * Concatenate the resulting list into a single cityblock distance matrix.

Note that concatenation is not a fast operation, so probably we will have to continue improving our function.

In [4]:
# define the list of operations to be performed asynchronously
chunks = 12  # we choose on chunk for physical cpu (gpu partition)
partial_distances = [cdist(x, xi, 'cityblock') for xi in np.split(x, chunks)]

In [5]:
# visualize the computational graph until this point

In [6]:
cbdm = np.concatenate(partial_distances, axis=1)

In [7]:
# visualize the computational graph

At this point, you should have the computational graph already defined. Let's run and time the compute step. We may go a shell and run the command `top`. Now you should see that the computation is executed in parallel resulting in a shorter execution time.

In [8]:
#time and run the computational graph

In [9]:
# check that the resulting matrices are the same
np.abs(cbdm - cdist(x, x, 'cityblock')).max()

0.0

A problem with this solution, as mentioned above, is that `np.concatenate` is not  a fast operation.
Let's check how much time it takes without the concatenation part:

In [10]:
# time and run the computational graph without the concatenate part

Let's implement the whole thing as a single function:

In [11]:
# Implementing the whole thing as a single function
def cityblock_dask_concat(x, y, chunks):
    """Implementation using array concatenation"""

In [13]:
# check that the resulting matrices are the same
# print(np.abs(cityblock_dask_concat(x, x, chunks) - cdist(x, x, 'cityblock')).max())

# Questions
 * Why is relevant for this implementation that `scipy.spatial.distance.cdist` is not multithreaded?